# Implementing your own model

In this tutorial we show how to implement your own model and test it on a dataset. 

This particular example uses the MUTAG dataset, uses an hypergraph lifting to create hypergraphs, and defines a model to work on them. 

We train the model using the appropriate training and validation datasets, and finally test it on the test dataset.

### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
&emsp;[1. Imports](##sec1)

&emsp;[2. Configurations and utilities](##sec2)

&emsp;[3. Loading the data](##sec3)

&emsp;[4. Backbone definition](##sec4)

&emsp;[5. Model initialization](##sec5)

&emsp;[6. Training](##sec6)

&emsp;[7. Testing the model](##sec7)

## 1. Imports <a class="anchor" id="sec1"></a>

In [2]:
import lightning as pl
import torch
from omegaconf import OmegaConf

from topobenchmarkx.data.loaders import  MoleculeDatasetLoader2, TUDatasetLoader
from topobenchmarkx.data.preprocessor import PreProcessor
from topobenchmarkx.dataloader import TBXDataloader
from topobenchmarkx.evaluator import TBXEvaluator
from topobenchmarkx.loss import TBXLoss
from topobenchmarkx.model import TBXModel
from topobenchmarkx.nn.backbones.simplicial import SANN
from topobenchmarkx.nn.wrappers.simplicial import SANNWrapper
from topobenchmarkx.nn.encoders import SANNFeatureEncoder
from topobenchmarkx.nn.readouts.sann import SANNReadout
from topobenchmarkx.optimizer import TBXOptimizer

%load_ext autoreload
%autoreload 2

## 2. Configurations and utilities <a class="anchor" id="sec2"></a>

Configurations can be specified using yaml files or directly specified in your code like in this example.

In [6]:
in_channels = 11
hidden_channels = 2*32
out_channels = 1
complex_dim = 2
max_hop = 2
n_layers = 4

loader_config = {
    "data_domain": "graph",
    "data_type": "QM9",
    "data_name": "QM9Custom",
    "data_dir": "./datasets/graph/QM9Custom",
    "max_ring_size": 3
}

transform_config = {
"clique_lifting": {
        "transform_type": "lifting",
        "transform_name": "SimplicialCliqueLifting",
        "feature_lifting": "Duplicate",
        "complex_dim": complex_dim
    },
'only_alpha': {
        "transform_type": "data manipulation",
        "transform_name": "KeepSelectedTargetIndices",
        "target_indices": [0]
    },
    "add_gpse_information": {
        "transform_type": "data manipulation",
        "transform_name": "AddGPSEInformation",
        "max_hop": max_hop,
        "max_rank": complex_dim,
        "copy_initial": True,
        "neighborhoods": ['0_incidence_0'],
        "device": "cpu",
        "pretrain_model": "ZINC",
        "dim_target_node": 51,
        "dim_target_graph": 11,
        "dim_out": 11
    },
}

wrapper_config = {
    "out_channels": hidden_channels,
    "num_cell_dimensions": in_channels,
}

split_config = {
    "learning_setting": "inductive",
    "split_type": "random",
    "data_seed": 0,
    "data_split_dir": "./datasets/data_splits/QM9Custom/",
    "k": 10,
    "train_prop": 0.5
}


readout_config = {
    "readout_name": "SANNReadout",
    "num_cell_dimensions": in_channels,
    "hidden_dim": hidden_channels,
    "out_channels": out_channels,
    "complex_dim": complex_dim,
    "max_hop": max_hop,
    "task_level": "graph",
    "pooling_type": "sum",
}

loss_config = {
    'dataset_loss': {"task": "regression", "loss_type": "mse"}
    }

evaluator_config = {"task": "regression",
                    "num_classes": out_channels, # If it's 1 we sum one so it works with TBXEvaluator
                    "metrics": ["mse", "mae"]}

optimizer_config = {"optimizer_id": "Adam",
                    "parameters":
                        {"lr": 1e-3,"weight_decay": 1e-4}
                    }

loader_config = OmegaConf.create(loader_config)
transform_config = OmegaConf.create(transform_config)
split_config = OmegaConf.create(split_config)
readout_config = OmegaConf.create(readout_config)
loss_config = OmegaConf.create(loss_config)
evaluator_config = OmegaConf.create(evaluator_config)
optimizer_config = OmegaConf.create(optimizer_config)

In [7]:
def wrapper(**factory_kwargs):
    def factory(backbone):
        return SANNWrapper(backbone, **factory_kwargs)
    return factory

## 3. Loading the data <a class="anchor" id="sec3"></a>

In this example we use the MUTAG dataset. It is a graph dataset and we use the k-hop lifting to transform the graphs into hypergraphs. 

We invite you to check out the README of the [repository](https://github.com/pyt-team/TopoBenchmarkX) to learn more about the various liftings offered.

In [8]:
graph_loader = MoleculeDatasetLoader2(loader_config)

dataset, dataset_dir = graph_loader.load()

preprocessor = PreProcessor(dataset, dataset_dir, transform_config, force_reload=True)
dataset_train, dataset_val, dataset_test = preprocessor.load_dataset_splits(split_config)
datamodule = TBXDataloader(dataset_train, dataset_val, dataset_test, batch_size=32)

Extracting datasets/graph/QM9Custom/raw/qm9.zip
Processing...
100%|██████████| 133885/133885 [06:37<00:00, 336.46it/s] 
Done!
Processing...
/Users/martincarrasco/miniconda3/envs/sann_311/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Done!


In [9]:
y_index = datamodule.dataset_train[0][1].index('y')
datamodule.dataset_train[0][0][y_index].shape

torch.Size([1])

## 4. Backbone definition <a class="anchor" id="sec4"></a>

To implement a new model we only need to define the forward method.

With a hypergraph with $n$ nodes and $m$ hyperedges this model simply calculates the hyperedge features as $X_1 = B_1 \cdot X_0$ where $B_1 \in \mathbb{R}^{n \times m}$ is the incidence matrix, where $B_{ij}=1$ if node $i$ belongs to hyperedge $j$ and is 0 otherwise.

Then the outputs are computed as $X^{'}_0=\text{ReLU}(W_0 \cdot X_0 + B_0)$ and $X^{'}_1=\text{ReLU}(W_1 \cdot X_1 + B_1)$, by simply using two linear layers with ReLU activation.

## 5. Model initialization <a class="anchor" id="sec5"></a>

Now that the model is defined we can create the TBXModel, which takes care of implementing everything else that is needed to train the model. 

First we need to implement a few classes to specify the behaviour of the model.

In [10]:
datamodule = TBXDataloader(dataset_train, dataset_val, dataset_test, batch_size=64)
backbone = SANN(in_channels=hidden_channels, hidden_channels=hidden_channels, n_layers=n_layers, update_func="lrelu", max_hop=2, layer_norm=True)
backbone_wrapper = wrapper(**wrapper_config)

readout = SANNReadout(**readout_config)
loss = TBXLoss(**loss_config)

in_channels_hops = [
    [in_channels, 62],
    [in_channels, 62],
    [in_channels, 62]
]
feature_encoder = SANNFeatureEncoder(in_channels=in_channels_hops, out_channels=hidden_channels, max_hop=2)

evaluator = TBXEvaluator(**evaluator_config)
optimizer = TBXOptimizer(**optimizer_config)

Now we can instantiate the TBXModel.

In [11]:
model = TBXModel(backbone=backbone,
                 backbone_wrapper=backbone_wrapper,
                 readout=readout,
                 loss=loss,
                 feature_encoder=feature_encoder,
                 evaluator=evaluator,
                 optimizer=optimizer,
                 compile=False)

## 6. Training <a class="anchor" id="sec6"></a>

Now we can use the `lightning` trainer to train the model.

In [16]:
# Increase the number of epochs to get better results
trainer = pl.Trainer(max_epochs=500, accelerator="cpu", enable_progress_bar=True, log_every_n_steps=1)

trainer.fit(model, datamodule)
train_metrics = trainer.callback_metrics

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name            | Type               | Params
-------------------------------------------------------
0 | feature_encoder | SANNFeatureEncoder | 14.4 K
1 | backbone        | SANNWrapper        | 101 K 
2 | readout         | SANNReadout        | 24.8 K
3 | val_acc_best    | MeanMetric         | 0     
-------------------------------------------------------
140 K     Trainable params
0         Non-trainable params
140 K     Total params
0.562     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.


In [17]:
print('      Training metrics\n', '-'*26)
for key in train_metrics:
    print('{:<21s} {:>5.4f}'.format(key+':', train_metrics[key].item()))

      Training metrics
 --------------------------
train/mae:            0.0543
train/mse:            0.0056
val/loss:             0.9067
val/mae:              0.7230
val/mse:              0.9811
train/loss:           0.0062


## 7. Testing the model <a class="anchor" id="sec7"></a>

Finally, we can test the model and obtain the results.

In [18]:
trainer.test(model, datamodule)
test_metrics = trainer.callback_metrics

/Users/martincarrasco/miniconda3/envs/sann_311/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/loss         │     7.603109359741211     │
│         test/mae          │    1.0013303756713867     │
│         test/mse          │     3.339362859725952     │
└───────────────────────────┴───────────────────────────┘

In [19]:
print('      Testing metrics\n', '-'*25)
for key in test_metrics:
    print('{:<20s} {:>5.4f}'.format(key+':', test_metrics[key].item()))

      Testing metrics
 -------------------------
test/loss:           7.6031
test/mae:            1.0013
test/mse:            3.3394
